# Dutch Cryptic Crosswords with Reasoning models

How good are reasoning models in solving cryptic crosswords? 

We use the code in this repo to give it a test.

This notebook will be updated with newer models.

#### Contents
0. Imports and settings
1. Load and inpsect dataset
2. Call OpenAI o1-preview

## 0. Imports and settings

In [2]:
import datasets
import openai
import os

In [ ]:
#Initialize the OpenAI client 
import config

os.environ['OPENAI_API_KEY'] = config.openai_key # replace with your API-key, but don't keep it in your source code :-)

client = openai.Client()

## 1. Load and inspect dataset

In [ ]:
from datasets import load_dataset

dataset = load_dataset("MichielBontenbal/DutchCrypticCrossword", split='train')
dataset

In [ ]:
print(dataset['vraag'][0])
print(dataset['antwoord'][0])

## 2. Call OpenAI o1-preview


In [6]:
#load a function
def call_openai_function(client, prompt, model="o1-preview"):
    return client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": prompt,
            }
        ],
        model=model,
    )

In [7]:
#setting the system prompt
system_prompt = 'Je bent een expert in het beantwoorden van cryptogrammen. Beantwoord de vraag van het cryptogram. Tussen de haakjes staat het aantal letters van het antwoord. Een voorbeeld: (5) betekent dat het antwoord 5 letters heeft. Geef alleen het antwoord geen andere tekst. De vraag is: '


In [ ]:
openai_o1_list = []
for i in range(len(dataset['vraag'])):
    promptvraag = dataset['vraag'][i]
    prompt = system_prompt + promptvraag
    chat_completion = call_openai_function(client, prompt, model="o1-preview")
    print(chat_completion.choices[0].message.content)
    openai_o1_list.append(chat_completion.choices[0].message.content)
    

In [ ]:
for i in range(len(dataset['vraag'])):
    print(dataset['antwoord'][i])

In [ ]:
for i in range(len(openai_o1_list)):
    if openai_o1_list[i].lower() == dataset['antwoord'][i].lower():
        print('yes')
    else:
        print('no')

In [ ]:
my_dict = dataset[0:86]
my_dict['antwoord']


In [28]:
my_dict["response"] = openai_o1_list
#my_dict = my_dict.assign(response=openai_o1_list)
#my_dict.add_column("response", openai_o1_list)

In [ ]:
my_dict['response']

In [ ]:
import json

json_string = json.dumps(my_dict)
print(json_string)
with open('openai_o1_20250103.json', 'w') as outfile:
    json.dump(my_dict, outfile, indent=4)

## 3. To do: Call QwQ 01